In [1]:
library(Seurat)
library(enrichR)
library(ggplot2)
library(dplyr)
library(stringr)
library(httr)
library(jsonlite)
library(hash)
library(readr)
library(purrr)
library(RColorBrewer)  # For color palettes
library(DESeq2)
library(ggpubr)


The legacy packages maptools, rgdal, and rgeos, underpinning the sp package,
which was just loaded, will retire in October 2023.
Please refer to R-spatial evolution reports for details, especially
https://r-spatial.org/r/2023/05/15/evolution4.html.
It may be desirable to make the sf package available;
package maintainers should consider adding sf to Suggests:.
The sp package is now running under evolution status 2
     (status 2 uses the sf package in place of rgdal)

Attaching SeuratObject

Welcome to enrichR
Checking connection ... 

Enrichr ... 
Connection is Live!

FlyEnrichr ... 
Connection is Live!

WormEnrichr ... 
Connection is Live!

YeastEnrichr ... 
Connection is Live!

FishEnrichr ... 
Connection is Live!

OxEnrichr ... 
Connection is Live!


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


hash-2.2.6.3 provided by Decision P

In [2]:
samples <- c("ZWV08", "ZWV10", "ZWV14","ZWV16")
scRNA <- c('Du13','Du27', 'Du53', 'Du55')

durva <- readRDS("durva_subset_roi.rds")
durva[["CellName"]] <- colnames(durva)


In [3]:
durva

An object of class Seurat 
66054 features across 29597 samples within 3 assays 
Active assay: integrated (2000 features, 2000 variable features)
 2 other assays present: RNA, SCT
 2 dimensional reductions calculated: pca, umap

In [4]:
minFeature<-200
maxFeature<- 9000
minCount<- 800
maxCount<- 40000
maxMT<-15
durva <- subset(durva, subset = nFeature_RNA > minFeature & nFeature_RNA < maxFeature & nCount_RNA > minCount &
                nCount_RNA < maxCount & percent.mt < maxMT & predicted_doublets ==F)


In [5]:
# Step 1: Split the durva object by 'orig.ident'
durva.list <- SplitObject(durva, split.by = "orig.ident")

# Step 2: Select integration features
features_durva <- SelectIntegrationFeatures(object.list = durva.list)

# Step 3: Preprocess each dataset (ScaleData and RunPCA)
durva.list <- lapply(X = durva.list, FUN = function(x) {
  x <- ScaleData(x, features = features_durva, verbose = FALSE)
  x <- RunPCA(x, features = features_durva, verbose = FALSE)
  return(x)
})

# Step 4: Find integration anchors using RPCA
durva.anchors <- FindIntegrationAnchors(object.list = durva.list, reduction = "rpca", dims = 1:30)

# Step 5: Integrate the datasets
durva.integrated <- IntegrateData(anchorset = durva.anchors, dims = 1:30)


Computing 2000 integration features

Scaling features for provided objects

Computing within dataset neighborhoods

Finding all pairwise anchors

Projecting new data onto SVD

Projecting new data onto SVD

Finding neighborhoods

Finding anchors

	Found 1131 anchors

Projecting new data onto SVD

Projecting new data onto SVD

Finding neighborhoods

Finding anchors

	Found 1571 anchors

Projecting new data onto SVD

Projecting new data onto SVD

Finding neighborhoods

Finding anchors

	Found 3048 anchors

Projecting new data onto SVD

Projecting new data onto SVD

Finding neighborhoods

Finding anchors

	Found 1554 anchors

Projecting new data onto SVD

Projecting new data onto SVD

Finding neighborhoods

Finding anchors

	Found 1521 anchors

Projecting new data onto SVD

Projecting new data onto SVD

Finding neighborhoods

Finding anchors

	Found 2530 anchors

Merging dataset 2 into 3

Extracting anchors for merged samples

Finding integration vectors

Finding integration vector weights

In [7]:
saveRDS(durva.integrated, file = "integrated_data/durva_subset_reintegrate.rds")


In [12]:
unique(durva.integrated@meta.data$orig.ident)

[1] "Du13" "Du53" "Du55" "Du27"

In [9]:
durva.integrated

An object of class Seurat 
66054 features across 23838 samples within 3 assays 
Active assay: integrated (2000 features, 2000 variable features)
 2 other assays present: RNA, SCT

In [10]:
durva

An object of class Seurat 
66054 features across 23838 samples within 3 assays 
Active assay: integrated (2000 features, 2000 variable features)
 2 other assays present: RNA, SCT
 2 dimensional reductions calculated: pca, umap